In [ ]:
from calendar import monthrange
import datetime

import altair as alt
import pandas as pd

- https://docs.python.org/3.7/library/calendar.html
- https://altair-viz.github.io/user_guide/generated/core/altair.TimeInterval.html
- https://vega.github.io/vega-lite/docs/datetime.html
- https://vega.github.io/vega-lite/docs/scale.html#example-customizing-domain-for-a-time-scale
- https://github.com/altair-viz/altair/issues/187
- https://github.com/d3/d3-time-format
- https://altair-viz.github.io/user_guide/times_and_dates.html
- https://vega.github.io/vega/docs/expressions/#datetime-functions
- https://stackoverflow.com/questions/60331614/how-to-highlight-a-bar-by-datetime-value-with-altair

In [ ]:
def get_date_end_month(year: int, month: int) -> str:
    # month: 1–12
    days = monthrange(year, month)[1]

    return f"{year}-{month:02}-{days}"

In [ ]:
milestones = pd.DataFrame(
    [
        {
            "milestone": "Phase I",
            "start": "2020-02-01",
            "end": get_date_end_month(2020, 7),
        },
        {
            "milestone": "Phase II",
            "start": "2020-10-01",
            "end": get_date_end_month(2021, 1),
        },
        {
            "milestone": "Phase III",
            "start": "2021-03-01",
            "end": get_date_end_month(2021, 6),
        },
        {
            "milestone": "Phase IV",
            "start": "2021-08-01",
            "end": get_date_end_month(2022, 1),
        },
    ]
)

In [ ]:
milestones

In [ ]:
time_domain = [{"year": 2020, "month": 1}, {"year": 2022, "month": 2}]

bar_height = 5
line_height = 1

# min_max = pd.DataFrame([{"start": milestones["start"].iloc[0], "end": milestones["end"].iloc[-1]}])
min_max = pd.DataFrame([{"start": "2020-01-01", "end": "2022-01-01"}])

gray = "#595959"
accent = "#3da1da"

In [ ]:
line = (
    alt.Chart(min_max, height=line_height)
    .mark_bar(size=line_height, color=gray)
    .encode(
        x=alt.X("start:T", scale=alt.Scale(domain=time_domain), axis=None),
        x2=alt.X2("end:T"),
    )
)
line

In [ ]:
formatter = "%b %Y"
labelExpr = "datum.label === 'Feb 2020' ? ['(Start)', datum.label] : datum.label === 'Jan 2022' ? [datum.label, '(End)'] : timeFormat(datum.value, '%Y')"
labelAlignExpr = "datum.label === 'Feb 2020' ? 'left' : datum.label === 'Jan 2022' ? 'right' : 'center'"

phase = "Phase I"

bars = (
    alt.Chart(milestones, height=bar_height)
    .mark_bar(tooltip=True, size=bar_height)
    .encode(
        x=alt.X(
            "start:T",
            scale=alt.Scale(domain=time_domain),
            axis=alt.Axis(
                title=None,
                domain=False,
                grid=False,
                values=["2020-02-01", "2021-01-01", "2022-01-31"],
                format=formatter,
                labelExpr=labelExpr,
                labelAlign={"expr": labelAlignExpr},
                labelFont="Open Sans",
                tickColor=gray,
                offset=3,
                tickSize=10,
                tickCap="round",
            ),
        ),
        x2=alt.X2("end:T"),
        color=alt.condition(
            alt.datum.milestone == phase, alt.value(accent), alt.value(gray)
        ),
    )
)

bars

In [ ]:
(line + bars).properties(
    usermeta={
        "embedOptions": {
            "downloadFileName": "gantt_chart",
            "scaleFactor": 5,
            "renderer": "svg",
        }
    }
).resolve_scale(x="independent").configure_view(strokeWidth=0)

---